In [1]:
import pandas as pd
import numpy as np

total = 1_427_843
total_ac = 625_781
total_nac = 802_062

In [2]:
f = '../data/002_intermediate/vivienda_energia_m2_estrato_clima.parquet'
vivienda = pd.read_parquet(f)
print("se tiran ",vivienda.m2.isna().sum())
print("Iniciales ",vivienda.factor.sum())
vivienda.dropna(inplace=True)
print("Restan    ",vivienda.factor.sum())


se tiran  51
Iniciales  1036944
Restan     951967


In [3]:
f = "../data/001_raw/ENCEVI_2018/aireacond.csv"
aire = pd.read_csv(f,index_col=0)
folios_con_ac = aire.index.unique()

# 2. Subconjunto de viv con AC
viv_con_ac = vivienda.loc[ vivienda.index.isin(folios_con_ac) ]

# 3. Subconjunto de viv sin AC
viv_sin_ac = vivienda.loc[ ~vivienda.index.isin(folios_con_ac) ]

# 4. Comprobar tamaños
print("Con AC:", viv_con_ac.factor.sum())
print("Sin AC:", viv_sin_ac.factor.sum())

Con AC: 419033
Sin AC: 532934


In [4]:
viv_con_ac.to_parquet("../data/002_intermediate/vivienda_con_aa.parquet")
viv_sin_ac.to_parquet("../data/002_intermediate/vivienda_sin_aa.parquet")

In [5]:
ac_mean = np.average(viv_con_ac['kwh_mes'],weights=viv_con_ac['factor'])
nac_mean = np.average(viv_sin_ac['kwh_mes'],weights=viv_sin_ac['factor'])

In [6]:
ac_mean, nac_mean

(np.float64(159.55143774011103), np.float64(120.54649414890874))

In [7]:
ac_energia = ac_mean*total_ac
nac_energia = nac_mean*total_nac
ac_energia, nac_energia

(np.float64(99844258.26044442), np.float64(96685762.19006203))

In [8]:
total_energia = ac_energia + nac_energia
total_energia

np.float64(196530020.45050645)

In [9]:
# Porcentaje de viviendas
porc_ac = total_ac / total * 100
porc_nac = total_nac / total * 100


# Porcentaje del consumo
porc_consumo_ac = ac_energia / total_energia * 100
porc_consumo_nac = nac_energia / total_energia * 100

# Crear tabla
import pandas as pd

tabla = pd.DataFrame({
    'Tipo de vivienda': ['Con AC', 'Sin AC'],
    'Viviendas (%)': [porc_ac, porc_nac],
    'Consumo (%)': [porc_consumo_ac, porc_consumo_nac]
})

print(tabla)


  Tipo de vivienda  Viviendas (%)  Consumo (%)
0           Con AC      43.827017    50.803566
1           Sin AC      56.172983    49.196434


In [10]:
viv_con_ac

,clima,factor,entidad,personas,m2,estrato,kwh_mes,kwh_m2,kwh_persona,kwh_m2persona
folio,,,,,,,,,,
16082,Cálido extremoso,1778,19,4,100.0,Medio bajo,91.972790,1.778141,44.453515,0.444535
16087,Cálido extremoso,1413,19,2,100.0,Medio alto,770.600719,7.192273,359.613669,3.596137
16088,Cálido extremoso,1413,19,2,100.0,Medio alto,47.375451,0.915925,45.796270,0.457963
16089,Cálido extremoso,1413,19,5,150.0,Medio alto,213.275324,2.796276,83.888294,0.559255
16090,Cálido extremoso,1413,19,2,100.0,Medio alto,91.219720,1.946021,97.301035,0.973010
...,...,...,...,...,...,...,...,...,...,...
16943,Cálido extremoso,940,19,5,55.0,Bajo,58.144420,2.149582,23.645397,0.429916
16945,Cálido extremoso,940,19,3,100.0,Bajo,129.134483,2.410510,80.350345,0.803503
16951,Cálido extremoso,940,19,3,100.0,Bajo,126.895967,2.537919,84.597311,0.845973


In [17]:
df = viv_con_ac[['m2','estrato','kwh_mes','factor']].copy()
df = df.dropna(subset=['kwh_mes','factor'])
df = df[df['factor'] > 0]

# Consumo ponderado por registro (kWh/mes * peso)
df['consumo_pond'] = df['kwh_mes'] * df['factor']

# Agregado por m2 y estrato
res = (
    df.groupby(['m2','estrato'], as_index=False)
      .agg(consumo_total_pond=('consumo_pond','sum'),
           peso=('factor','sum'),
           n_filas=('kwh_mes','size'))
)

# Porcentaje respecto al consumo total (suma ponderada global)
total_consumo = res['consumo_total_pond'].sum()
res['pct_consumo_total'] = 100 * res['consumo_total_pond'] / total_consumo

# Orden y redondeo para presentación
res = res.sort_values(['estrato','m2']).reset_index(drop=True)
res['consumo_total_pond'] = res['consumo_total_pond'].round(2)
res['pct_consumo_total'] = res['pct_consumo_total'].round(2)

res


,m2,estrato,consumo_total_pond,peso,n_filas,pct_consumo_total
0,55.0,Alto,404776.30,5232,3,0.61
1,75.0,Alto,1587764.40,11841,7,2.37
2,100.0,Alto,8105589.07,59321,37,12.12
3,150.0,Alto,7213327.21,43246,27,10.79
4,200.0,Alto,5224366.26,19064,11,7.81
5,55.0,Bajo,87317.15,1641,2,0.13
6,75.0,Bajo,168482.71,2581,3,0.25
7,100.0,Bajo,566226.16,2820,3,0.85
8,200.0,Bajo,131359.10,743,1,0.20
9,55.0,Medio alto,372696.20,4560,3,0.56
